<a href="https://colab.research.google.com/github/TheViet298/BTL_NMKHDL/blob/main/BTL_NMKHDL_N5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
# Cài đặt thư viện cần thiết
!pip3 install pandas;
!pip3 install tqdm;
!pip3 install bs4;
!pip3 install regex;
!pip3 install numpy;
!pip3 install pyvi;
!pip3 install gensim;
!pip3 install matplotlib;
!pip3 install seaborn;
!pip3 install -U scikit-learn;
!pip3 install lime;
!pip3 install

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 125

import seaborn as sns

import pandas as pd
import numpy as np
np.random.seed(0) # Chạy các hàm của scikit-learn giống nhau mỗi lần chạy

import regex as re
import time
from tqdm.notebook import tqdm # Hiện thanh progress cho đẹp :D
tqdm.pandas()

# Thư viện để request và parse HTML
import requests
from bs4 import BeautifulSoup

# Các thư viện liên quan tới ngôn ngữ và NLP
from pyvi import ViTokenizer
import gensim
import unicodedata

# Trực quan hóa mô hình dự đoán văn bản
from lime import lime_text

# Lưu lại model
import pickle

# Thư viện liên quan của Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay # vẽ ma trận nhầm lẫn
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import feature_selection

# Tạo pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, make_column_selector

# Các mô hình học
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier # Phương pháp bagging

In [5]:
# Thiết lập đường dẫn cho phần 1
dir_1 = "src/scraped_data/"

In [6]:
# Thử nghiệm url đầu tiên
url = "https://tuoitre.vn/tin-moi-nhat.htm"

# Lấy danh sách box-category-item
html_text = requests.get(url).text
html_tree = BeautifulSoup(html_text, 'html.parser')
news_items = html_tree.find_all('div', class_='box-category-item')

# tạo df
data = []

# Lấy ra các trường từ box-category-item
for item in news_items:
  title_t = item.find('a', class_='box-category-link-title') # Tiêu đề
  category_t = item.find('a', class_= 'box-category-category') # Thể loại
  description_t = item.find('div', class_= 'box-category-content') # mô tả

  # Kiểm tra nếu tìm thấy các trường
  if title_t and category_t:
    title = title_t.text.strip()
    link = "https://tuoitre.vn" + title_t.get("href", "")
    category = category_t.text.strip()
    # thêm dữ liệu các trường vào data
    data.append({"links": link, "title": title,"description": "Null","content": "Null", "class": category})

# Chuyển danh sách thành DataFrame
raw_data = pd.DataFrame(data)

raw_data.tail()
# print(raw_data.head())

,links,title,description,content,class
15,https://tuoitre.vn/diem-tin-8h-gia-vang-duoc-d...,Điểm tin 8h: Giá vàng được dự báo giảm trong t...,Null,Null,Cần biết
16,https://tuoitre.vn/dong-tay-land-bat-tay-ecopa...,Đông Tây Land bắt tay Ecopark lan tỏa Eco Retreat,Null,Null,Kinh doanh
17,https://tuoitre.vn/khach-hang-can-ngan-hang-gi...,Khách hàng cần ngân hàng giúp khai phóng sức m...,Null,Null,Kinh doanh
18,https://tuoitre.vn/khong-gian-song-cho-cong-da...,Không gian sống cho công dân toàn cầu tại Crys...,Null,Null,Dự án
19,https://tuoitre.vn/lam-sao-phat-hien-o-nhiem-k...,Làm sao phát hiện ô nhiễm không khí bằng mắt t...,Null,Null,Sức khỏe


In [7]:
%%time
for _, row in raw_data.iterrows():
    try:
        # Gửi request để lấy nội dung trang bài viết
        news_page = requests.get(row["links"]).content
        news_tree = BeautifulSoup(news_page, "html.parser")

        # Lấy mô tả (description)
        try:
            row["description"] = news_tree.find("h2", class_="detail-sapo").text.strip()
        except:
            row["description"] = ''

        # Kiểm tra nếu bài viết là dạng "điểm tin" hoặc "video" (không có nội dung chính)
        body = news_tree.find("div", class_="detail-content afcbc-body readmore-body")

        if body:  # Nếu có thẻ nội dung thì lấy text
            content = body.find_all("p", recursive=False)
            row["content"] = "\n".join([x.text.strip() for x in content])
        else:  # Nếu không có thẻ nội dung chính, gán content là ''
            row["content"] = ''

    except requests.exceptions.RequestException as e:
        print(f"Lỗi request {row['links']}: {e}")
        row["description"], row["content"] = '', ''

    time.sleep(0.05)

# raw_data.tail()
print(raw_data.head())

                                               links  \
0  https://tuoitre.vn/thu-mon-ho-tung-han-bat-chi...   
1  https://tuoitre.vn/khong-co-bang-lai-con-di-ng...   
2  https://tuoitre.vn/lam-freelancer-hap-dan-that...   
3  https://tuoitre.vn/tran-thanh-luc-vo-dich-worl...   
4  https://tuoitre.vn/chung-luoi-ban-do-hay-gap-l...   

                                               title  \
0          Thủ môn Hồ Tùng Hân bắt chính ở Singapore   
1  Không có bằng lái còn đi ngược chiều cao tốc, ...   
2                 Làm freelancer hấp dẫn thật không?   
3  Trần Thanh Lực vô địch World Cup billiards Bogota   
4  Chứng 'lưỡi bản đồ' hay gặp là bệnh gì và có n...   

                                         description  \
0  Tối 2-3, thủ môn Hồ Tùng Hân lần đầu ra sân ch...   
1  Tài xế L.V.N. lái xe khách 29 chỗ chở khách đi...   
2  Freelancer vốn được hiểu là lao động làm việc ...   
3  Sáng 3-3, tại chung kết World Cup billiards ca...   
4  Lưỡi bản đồ là một tình trạng lành tính thư

In [8]:
raw_data.tail()

,links,title,description,content,class
15,https://tuoitre.vn/diem-tin-8h-gia-vang-duoc-d...,Điểm tin 8h: Giá vàng được dự báo giảm trong t...,Điểm tin cùng bạn 8h ngày 3-3-2025: Phát động ...,Điểm tin 8h,Cần biết
16,https://tuoitre.vn/dong-tay-land-bat-tay-ecopa...,Đông Tây Land bắt tay Ecopark lan tỏa Eco Retreat,Đông Tây Land chính thức hợp tác với nhà sáng ...,"Cú bắt tay tôn vinh giá trị sống khác biệt\n""Đ...",Kinh doanh
17,https://tuoitre.vn/khach-hang-can-ngan-hang-gi...,Khách hàng cần ngân hàng giúp khai phóng sức m...,Khi dòng tiền luôn chuyển động và được tối ưu ...,Nhân sự kiện Ngân hàng Quốc tế (VIB) ra mắt tà...,Kinh doanh
18,https://tuoitre.vn/khong-gian-song-cho-cong-da...,Không gian sống cho công dân toàn cầu tại Crys...,"Trong thời đại hội nhập và toàn cầu hóa, trở t...",Ngày càng nhiều bạn trẻ lựa chọn lối sống tự d...,Dự án
19,https://tuoitre.vn/lam-sao-phat-hien-o-nhiem-k...,Làm sao phát hiện ô nhiễm không khí bằng mắt t...,Ô nhiễm không khí tác hại rất lớn đến sức khỏe...,Thứ nhất là nhìn thấy bụi mù và khói. Không kh...,Sức khỏe


Vấn đề phát sinh trong quá trình thu thập dữ liệu tự động hàng loạt.
Thiếu dữ liệu làm đứt quãng quá trình thu thập.
Do khối lượng dữ liệu có thể thu thập được rất lớn (ước tính lên đến hơn 500.000 tin), việc giám sát quá trình khai thác liên tục là không thể. Việc xuất hiện lỗi này đã khiến quá trình khai thác bị ngắt quãng mỗi khi xuất hiện. Nhóm đã khắc phục bằng cách đặt các khối lệnh try catch ở mỗi đoạn code parse các phần thông tin, tự động trả về rỗng nếu có lỗi.

In [9]:

# Định nghĩa URL trang tin mới nhất
url = "https://tuoitre.vn/tin-moi-nhat.htm"

def fetch_latest_news():
    """
    Lấy danh sách bài viết từ trang: tin mới nhất của Tuổi Trẻ.
    """
    session = requests.Session()
    session.headers.update({'User-Agent': 'Mozilla/5.0'})

    response = session.get(url)
    response.raise_for_status()  # Kiểm tra lỗi HTTP

    html_tree = BeautifulSoup(response.text, 'html.parser')
    news_items = html_tree.find_all('div', class_='box-category-item')

    data = []
    for item in news_items:
        title_t = item.find('a', class_='box-category-link-title')
        category_t = item.find('a', class_='box-category-category')

        if title_t and category_t:
            title = title_t.text.strip()
            link = "https://tuoitre.vn" + title_t.get("href", "")
            category = category_t.text.strip()
            data.append({"links": link, "title": title, "class": category})

    df = pd.DataFrame(data)
    return df

# Kiểm tra dữ liệu
df_news = fetch_latest_news()
print(df_news.head(10))

# Lưu để kiểm tra
df_news.to_csv("check_news.csv", index=False, encoding="utf-8")

                                               links  \
0  https://tuoitre.vn/thu-mon-ho-tung-han-bat-chi...   
1  https://tuoitre.vn/khong-co-bang-lai-con-di-ng...   
2  https://tuoitre.vn/lam-freelancer-hap-dan-that...   
3  https://tuoitre.vn/tran-thanh-luc-vo-dich-worl...   
4  https://tuoitre.vn/chung-luoi-ban-do-hay-gap-l...   
5  https://tuoitre.vn/truong-thcs-thpt-tran-dai-n...   
6  https://tuoitre.vn/hlv-guardiola-chi-trich-qua...   
7  https://tuoitre.vn/khi-quan-he-my-va-chau-au-t...   
8  https://tuoitre.vn/thang-3-ve-ngam-hoa-giay-bu...   
9  https://tuoitre.vn/hai-kich-ban-chung-khoan-tu...   

                                               title          class  
0          Thủ môn Hồ Tùng Hân bắt chính ở Singapore       Thể thao  
1  Không có bằng lái còn đi ngược chiều cao tốc, ...      Pháp luật  
2                 Làm freelancer hấp dẫn thật không?  Nhịp sống trẻ  
3  Trần Thanh Lực vô địch World Cup billiards Bogota       Thể thao  
4  Chứng 'lưỡi bản đồ' hay gặp là

In [10]:
def fetch_article_details(link):
    """
    Lấy mô tả và nội dung bài viết từ đường dẫn.
    """
    session = requests.Session()
    session.headers.update({'User-Agent': 'Mozilla/5.0'})

    try:
        response = session.get(link)
        response.raise_for_status()
        news_tree = BeautifulSoup(response.content, "html.parser")

        # Lấy mô tả (description)
        description = news_tree.find("h2", class_="detail-sapo")
        description_text = description.text.strip() if description else ""

        # Lấy nội dung bài báo (content)
        body = news_tree.find("div", class_="detail-content afcbc-body readmore-body")
        if body:
            content = "\n".join([p.text.strip() for p in body.find_all("p", recursive=False)])
        else:
            content = ""

        return description_text, content

    except requests.exceptions.RequestException as e:
        print(f"Lỗi request {link}: {e}")
        return "", ""

# Test với 1 bài báo đầu tiên trong danh sách
test_link = df_news.iloc[0]["links"]
desc, content = fetch_article_details(test_link)
print("🔹 Mô tả:", desc)
print("🔹 Nội dung:", content[:500]) # lấy 500 ký tự thôi


🔹 Mô tả: Tối 2-3, thủ môn Hồ Tùng Hân lần đầu ra sân cho CLB Balestier Khalsa (Singapore) trong trận đấu gặp Pathum United (Thái Lan) tại lượt hai vòng bảng Singapore Cup 2025.
🔹 Nội dung: Sau 1 tháng tập luyện, thủ môn Hồ Tùng Hân được ban huấn luyện CLB Balestier Khalsa trao cơ hội bắt chính đầu tiên ở đấu trường Singapore Cup, khi gặp đại diện Thái Lan Pathum United ở lượt hai bảng B trên sân nhà.
Tùng Hân chơi trọn vẹn 90 phút, ghi dấu ấn với một pha bắt phạt đền ở phút 45+2. Tuy nhiên sau khi Hân đẩy được bóng, các đồng đội của anh không kịp hỗ trợ và để cho Nattawut Suksum đá bồi nâng tỷ số lên 2-1 cho Pathum Utd.
Trước đó, Anton Fase ghi bàn mở tỷ số cho Balestier Khalsa ở 


In [11]:

url = "https://tuoitre.vn/timeline/0/trang-1.htm"
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open("tuoitre_page1.html", "w", encoding="utf-8") as f:
        f.write(response.text)
    print("✅ Đã lưu HTML trang 1. Hãy mở file 'tuoitre_page1.html' để kiểm tra.")
else:
    print(f"❌ Lỗi: {response.status_code}")


✅ Đã lưu HTML trang 1. Hãy mở file 'tuoitre_page1.html' để kiểm tra.


In [12]:

def single_request_scraping(index, limit_retry=10, sleep_time=0.1):
    """
    Lấy 20 bài báo từ trang `index` của Tuổi Trẻ.
    """
    session = requests.Session()
    session.headers.update({'User-Agent': 'Mozilla/5.0'})

    url = f"https://tuoitre.vn/timeline/0/trang-{index}.htm"
    allow_status = [200, 301]

    try:
        response = session.get(url)
        try_left = limit_retry
        while response.status_code not in allow_status and try_left > 0:
            print(f"❌ Lỗi {response.status_code} tại trang {index}, thử lại...")
            response = session.get(url)
            try_left -= 1
        if response.status_code not in allow_status:
            print(f"❌ Bỏ qua trang {index} do lỗi HTTP {response.status_code}")
            return None

        html_tree = BeautifulSoup(response.text, 'html.parser')
        news_items = html_tree.find_all('div', class_='box-category-item')

        if not news_items:
            print(f"⚠️ Trang {index} không có bài báo nào.")
            return None

        data = []
        for item in news_items:
            try:
                # Lấy tiêu đề và link bài báo
                title_tag = item.find('a', class_='box-category-link-title')
                title = title_tag.text.strip()
                link = "https://tuoitre.vn" + title_tag.get("href", "")

                # Lấy danh mục bài báo
                category_tag = item.find('a', class_='box-category-category')
                category = category_tag.text.strip() if category_tag else "Không rõ"

                # Lấy mô tả ngắn (sapo)
                sapo_tag = item.find('p', class_='box-category-sapo')
                description = sapo_tag.text.strip() if sapo_tag else ""

                data.append({"links": link, "title": title, "description": description, "content": "", "class": category})
            except AttributeError:
                continue

        df = pd.DataFrame(data)

        # Tiến hành lấy nội dung từng bài báo
        for i, row in df.iterrows():
            try:
                article_response = session.get(row["links"])
                news_tree = BeautifulSoup(article_response.content, "html.parser")

                # Lấy nội dung bài báo
                body = news_tree.find("div", class_="detail-content")
                if body:
                    content = "\n".join([p.text.strip() for p in body.find_all("p", recursive=False)])
                    df.at[i, "content"] = content
            except Exception as e:
                print(f"❌ Lỗi khi lấy bài {row['links']}: {e}")

            time.sleep(sleep_time)  # Giảm nguy cơ bị chặn

        return df

    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi request trang {index}: {e}")
        return None


Check dữ liệu trang 1

In [13]:
df = single_request_scraping(1)  # Lấy dữ liệu trang 1
print(df.head())  # In 5 dòng đầu tiên


                                               links  \
0  https://tuoitre.vn/cong-dan-tre-tieu-bieu-nhi-...   
1  https://tuoitre.vn/diem-tin-8h-gia-vang-duoc-d...   
2  https://tuoitre.vn/dong-tay-land-bat-tay-ecopa...   
3  https://tuoitre.vn/khach-hang-can-ngan-hang-gi...   
4  https://tuoitre.vn/khong-gian-song-cho-cong-da...   

                                               title  \
0  Công dân trẻ tiêu biểu 'nhí' nhất: Đam mê toán...   
1  Điểm tin 8h: Giá vàng được dự báo giảm trong t...   
2  Đông Tây Land bắt tay Ecopark lan tỏa Eco Retreat   
3  Khách hàng cần ngân hàng giúp khai phóng sức m...   
4  Không gian sống cho công dân toàn cầu tại Crys...   

                                         description  \
0  Dù đang học lớp 6, cậu học trò Cáp Hoàng Dũng,...   
1  Nhiều thông tin được cập nhật trong chương trì...   
2  Đông Tây Land chính thức hợp tác với nhà sáng ...   
3  Khách hàng cần ngân hàng giúp khai phóng sức m...   
4  Trong thời đại hội nhập và toàn cầu hóa, tr

Lưu kết quả file csv

In [14]:
df.to_csv("tuoitre_data.csv", index=False, encoding="utf-8-sig")
print("✅ Đã lưu dữ liệu vào tuoitre_data.csv")


✅ Đã lưu dữ liệu vào tuoitre_data.csv


In [15]:
print(f"🔍 Tổng số bài báo thu thập: {len(df)}")

🔍 Tổng số bài báo thu thập: 21


In [16]:
print(df[df["content"].isna()])  # Hiển thị các bài báo không có nội dung


Empty DataFrame
Columns: [links, title, description, content, class]
Index: []


In [17]:
print(df["links"])

0     https://tuoitre.vn/cong-dan-tre-tieu-bieu-nhi-...
1     https://tuoitre.vn/diem-tin-8h-gia-vang-duoc-d...
2     https://tuoitre.vn/dong-tay-land-bat-tay-ecopa...
3     https://tuoitre.vn/khach-hang-can-ngan-hang-gi...
4     https://tuoitre.vn/khong-gian-song-cho-cong-da...
5     https://tuoitre.vn/lam-sao-phat-hien-o-nhiem-k...
6     https://tuoitre.vn/lisa-blackpink-ariana-grand...
7     https://tuoitre.vn/cai-nhau-chuyen-ben-tay-202...
8     https://tuoitre.vn/bo-y-te-de-xuat-quy-dinh-mo...
9     https://tuoitre.vn/kia-ev4-chinh-thuc-ra-mat-g...
10    https://tuoitre.vn/ong-zelensky-khang-dinh-ukr...
11    https://tuoitre.vn/le-trao-giai-oscar-2025-dan...
12    https://tuoitre.vn/tin-tuc-the-gioi-3-3-ong-ze...
13    https://tuoitre.vn/bat-ngo-vi-lam-ly-lich-tu-p...
14    https://tuoitre.vn/bong-da-tp-hcm-khong-co-kha...
15    https://tuoitre.vn/nha-gia-tien-thu-187-ti-don...
16    https://tuoitre.vn/fulham-bien-man-united-than...
17    https://tuoitre.vn/xuat-sac-danh-bai-co-th

Thử lại lấy toàn bộ trang trên tuoitre.vn Vì ở trên nó chỉ lấy được các bài báo trên 1 trang duy nhất

In [18]:

# Thiết lập đường dẫn cho phần 1
def single_request_scraping(index=1, limit_retry=100, sleep_time=0.05):
    '''
    Thu thập các trang tin trong mục tin mới nhất của báo Tuổi Trẻ.
    Mỗi lần thu thập 20 tin. Sử dụng số index để thu thập các trang liên tiếp.

    Param:
        index: số chỉ trang cần request.
    '''
    allow_status = [200, 301]
    s = requests.Session()
    s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    url = f"https://tuoitre.vn/timeline/0/trang-{index}.htm"

    # Lấy danh sách news_items
    try:
        response = s.get(url)
        try_left = limit_retry
        while response.status_code not in allow_status and try_left > 0:
            print(f"Lỗi {response.status_code} tại trang {index}")
            response = s.get(url)
            try_left -= 1
        html_text = response.text
    except:
        print(f"Lỗi request tại trang {index}")
        return None

    html_tree = BeautifulSoup(html_text, 'html.parser')
    news_items = html_tree.find_all('div', class_='box-category-item')

    # Kiểm tra lỗi nếu không lấy được bất cứ item nào
    if len(news_items) == 0:
        print(f'Trang {index} không lấy được item.')
        print(html_text)
        return None
    elif len(news_items) != 20:
        print(f"Warning: Trang {index} chỉ lấy được {len(news_items)} items.")

    # Lấy ra link, title và category từ news_items
    data_list = []  # Sử dụng danh sách để lưu dữ liệu trước khi tạo DataFrame
    for item in news_items:
        try:
            title_tag = item.find('a', class_='box-category-link-title')
            title = title_tag.text.strip() if title_tag else ""
            link = "https://tuoitre.vn" + title_tag["href"] if title_tag and title_tag.has_attr("href") else ""  # Lấy link nếu có
        except:
            title, link = "", ""

        try:
            category = item.find('a', class_='box-category-category').text.strip()
        except:
            category = ""

        data_list.append({"links": link, "title": title, "class": category, "description": "", "content": ""})

    # Chuyển danh sách thành DataFrame
    raw_data = pd.DataFrame(data_list)

    # Tiến hành lấy nội dung từng link
    for i, row in raw_data.iterrows():
        if not row["links"]:  # Bỏ qua nếu không có link
            continue

        try:
            response = s.get(row["links"])
            try_left = limit_retry
            while response.status_code not in allow_status and try_left > 0:
                print(f"Lỗi {response.status_code} tại link {row['links']}")
                response = s.get(row["links"])
                try_left -= 1
            news_page = response.content
        except:
            print(f"Lỗi request tại link {row['links']}")
            raw_data.at[i, "description"] = ""
            raw_data.at[i, "content"] = ""
            continue

        news_tree = BeautifulSoup(news_page, "html.parser")
        # Lấy mô tả
        try:
            raw_data.at[i, "description"] = news_tree.find("h2", class_="detail-sapo").text.strip()
        except:
            raw_data.at[i, "description"] = ''

        # Lấy nội dung bài viết
        body = news_tree.find("div", class_="detail-content afcbc-body readmore-body")
        if body:
            content = body.find_all("p", recursive=False)
            raw_data.at[i, "content"] = "\n".join([x.text.strip() for x in content])
        else:
            raw_data.at[i, "content"] = ''

        time.sleep(sleep_time)

    return raw_data

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import os
def batch_scraping(num=200, output_dir=""):
    '''
    Thực hiện thu thập toàn bộ dữ liệu trên trang tuoitre.vn

    Param:
        num: số lượng trang request cho 1 batch (1 trang = 20 bài báo). Mặc định là 200 trang (~4000 bài báo).
    '''
    error_links = []  # Danh sách lưu các link bị lỗi 502

    # Tạo thư mục output nếu chưa có
    os.makedirs(output_dir, exist_ok=True)

    # Checkpoint: Lưu tiến trình
    checkpoint_file = os.path.join(output_dir, "checkpoint.txt")

    # Đọc batch cuối cùng từ checkpoint
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r") as f:
            iter_num = int(f.read().strip())
        print(f"🔄 Tiếp tục từ batch {iter_num}")
    else:
        iter_num = 0

    while True:
        batch_df = pd.DataFrame(columns=["links", "title", "description", "content", "class"])

        for index in range(iter_num * num + 1, (iter_num + 1) * num + 1):
            retry_count = 3  # Số lần thử lại nếu lỗi 502

            while retry_count > 0:
                try:
                    data = single_request_scraping(index)

                    if data is None or data.empty:
                        print(f"❌ Không lấy được dữ liệu từ trang {index}, bỏ qua...")
                        break  # Bỏ qua nếu trang không có dữ liệu

                    print(f"✅ Page {index} complete!")
                    batch_df = pd.concat([batch_df, data], ignore_index=True)
                    break  # Thành công thì thoát vòng retry

                except Exception as e:
                    error_message = str(e)
                    if "502" in error_message:
                        print(f"⚠ Lỗi 502 tại trang {index}, thử lại ({retry_count} lần)...")
                        retry_count -= 1
                        time.sleep(2)  # Chờ 2 giây trước khi thử lại
                    else:
                        print(f"❌ Lỗi không xác định tại trang {index}: {error_message}")
                        return  # Dừng chương trình nếu lỗi nghiêm trọng

            if retry_count == 0:  # Nếu thử lại 3 lần vẫn lỗi 502 thì lưu vào log
                error_links.append(f"Page {index}: {error_message}")

        # Lưu batch nếu có dữ liệu
        if not batch_df.empty:
            output_file = os.path.join(output_dir, f'crawling_{iter_num}.csv')
            batch_df.to_csv(output_file, index=False, encoding="utf-8")
            print(f"📁 Batch {iter_num} đã được lưu: {output_file}")

            # Lưu checkpoint sau mỗi batch
            with open(checkpoint_file, "w") as f:
                f.write(str(iter_num))

        iter_num += 1

    # Ghi lại danh sách link lỗi 502 vào file log
    if error_links:
        with open(os.path.join(output_dir, "error_502.log"), "w", encoding="utf-8") as log_file:
            log_file.write("\n".join(error_links))
        print(f"📄 Danh sách lỗi 502 đã được lưu vào {output_dir}/error_502.log")


In [22]:
batch_scraping(output_dir="/src/scraped_data")

✅ Page 1 complete!
✅ Page 2 complete!
✅ Page 3 complete!
✅ Page 4 complete!
✅ Page 5 complete!
✅ Page 6 complete!
✅ Page 7 complete!
Lỗi 502 tại trang 8
✅ Page 8 complete!
✅ Page 9 complete!
✅ Page 10 complete!
✅ Page 11 complete!
✅ Page 12 complete!
✅ Page 13 complete!
✅ Page 14 complete!
✅ Page 15 complete!
✅ Page 16 complete!
✅ Page 17 complete!
✅ Page 18 complete!
✅ Page 19 complete!
✅ Page 20 complete!
✅ Page 21 complete!
✅ Page 22 complete!
✅ Page 23 complete!
✅ Page 24 complete!
✅ Page 25 complete!
✅ Page 26 complete!
✅ Page 27 complete!
✅ Page 28 complete!
✅ Page 29 complete!
✅ Page 30 complete!
✅ Page 31 complete!
✅ Page 32 complete!
✅ Page 33 complete!
✅ Page 34 complete!
✅ Page 35 complete!
✅ Page 36 complete!
✅ Page 37 complete!
✅ Page 38 complete!
✅ Page 39 complete!
✅ Page 40 complete!
Lỗi 502 tại link https://tuoitre.vn/dien-dan-tuong-lai-asean-2025-dau-an-moi-cua-ngoai-giao-viet-nam-20250227073719223.htm
✅ Page 41 complete!
✅ Page 42 complete!
✅ Page 43 complete!
✅ Page

KeyboardInterrupt: 